# Module 3: Risk Management

This notebook demonstrates risk management concepts, including Value at Risk (VaR) and portfolio risk simulation, using both sample and real financial data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

sns.set(style="whitegrid")

In [ ]:
np.random.seed(123)
returns = np.random.normal(loc=0.0005, scale=0.01, size=1000)
cumulative_returns = np.cumprod(1 + returns) - 1

plt.figure(figsize=(10, 5))
plt.plot(cumulative_returns, color='blue')
plt.title('Cumulative Returns of Simulated Asset')
plt.xlabel('Days')
plt.ylabel('Cumulative Returns')
plt.show()

In [ ]:
VaR_95 = np.percentile(returns, 5)
VaR_99 = np.percentile(returns, 1)
print(f"Value at Risk (95% confidence): {VaR_95:.4f}")
print(f"Value at Risk (99% confidence): {VaR_99:.4f}")

In [ ]:
# Simulate returns for two assets
returns_matrix = np.random.normal(loc=0.0005, scale=0.01, size=(1000, 2))
weights = np.array([0.6, 0.4])
portfolio_returns = returns_matrix @ weights
portfolio_VaR_95 = np.percentile(portfolio_returns, 5)
print(f"Portfolio Value at Risk (95% confidence): {portfolio_VaR_95:.4f}")

## Real-Time Example: Value at Risk for Real Stock Data

Let's fetch real stock data and calculate Value at Risk (VaR) for daily returns.

In [ ]:
data = yf.download('GOOGL', start='2022-01-01', end='2023-01-01')
data['Daily Return'] = data['Close'].pct_change().dropna()
real_VaR_95 = np.percentile(data['Daily Return'].dropna(), 5)
real_VaR_99 = np.percentile(data['Daily Return'].dropna(), 1)
print(f"GOOGL Value at Risk (95% confidence): {real_VaR_95:.4f}")
print(f"GOOGL Value at Risk (99% confidence): {real_VaR_99:.4f}")

plt.figure(figsize=(10, 5))
plt.hist(data['Daily Return'].dropna(), bins=50, color='skyblue', edgecolor='black')
plt.title('GOOGL Daily Returns Distribution')
plt.xlabel('Daily Return')
plt.ylabel('Frequency')
plt.show()

## Advanced Scenario: Conditional Value at Risk (CVaR)

CVaR (also known as Expected Shortfall) measures the expected loss in the worst-case (tail) scenarios beyond the VaR threshold.

In [ ]:
# CVaR for simulated returns
alpha = 0.05
VaR_level = np.percentile(returns, 100*alpha)
CVaR = returns[returns <= VaR_level].mean()
print(f"Simulated Asset CVaR (95%): {CVaR:.4f}")

# CVaR for real returns
daily_returns_real = data['Daily Return'].dropna()
VaR_level_real = np.percentile(daily_returns_real, 100*alpha)
CVaR_real = daily_returns_real[daily_returns_real <= VaR_level_real].mean()
print(f"GOOGL CVaR (95%): {CVaR_real:.4f}")

## Advanced Scenario: Monte Carlo Simulation for Portfolio Risk

Simulate thousands of possible future portfolio returns to estimate the distribution of outcomes and risk.

In [ ]:
# Monte Carlo simulation for a two-asset portfolio
num_simulations = 10000
simulated_portfolio_returns = []
for _ in range(num_simulations):
    sim_returns = np.random.normal(loc=0.0005, scale=0.01, size=(252, 2))
    sim_portfolio = sim_returns @ weights
    simulated_portfolio_returns.append(sim_portfolio.sum())
simulated_portfolio_returns = np.array(simulated_portfolio_returns)

plt.figure(figsize=(10, 5))
plt.hist(simulated_portfolio_returns, bins=50, color='orange', edgecolor='black')
plt.title('Monte Carlo Simulated Portfolio Returns (1 Year)')
plt.xlabel('Total Return')
plt.ylabel('Frequency')
plt.show()

# Estimate probability of loss
default_prob = np.mean(simulated_portfolio_returns < 0)
print(f"Probability of annual loss: {default_prob:.2%}")

## Advanced Scenario: Stress Testing

Stress testing evaluates portfolio performance under extreme but plausible market conditions.

In [ ]:
# Stress test: apply a -5% market shock to all returns for a week
shock = -0.05
shocked_returns = daily_returns_real.copy()
shocked_returns.iloc[:5] += shock

plt.figure(figsize=(10, 5))
plt.plot(daily_returns_real.index, daily_returns_real, label='Original Returns')
plt.plot(shocked_returns.index, shocked_returns, label='Shocked Returns', linestyle='--')
plt.title('Stress Test: Market Shock on GOOGL Returns')
plt.xlabel('Date')
plt.ylabel('Daily Return')
plt.legend()
plt.show()

# Compare mean returns before and after shock
print(f"Mean return before shock: {daily_returns_real.mean():.4f}")
print(f"Mean return after shock: {shocked_returns.mean():.4f}")